In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Imports here
%matplotlib inline
import numpy as np
import cv2
import os
import sys
import time
import json
import shutil

import PIL
import PIL.Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torch.autograd import Variable
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim

import torch
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.datasets import ImageFolder, DatasetFolder


from pathlib import Path


import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as T
import torchvision.utils as vutils

from typing import *
from PIL import Image

import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
class NormalizeInverse(T.Normalize):
    """
    Undoes the normalization and returns the reconstructed images in the input domain.
    """

    def __init__(self, mean, std):
        mean = torch.as_tensor(mean)
        std = torch.as_tensor(std)
        std_inv = 1 / (std + 1e-7)
        mean_inv = -mean * std_inv
        super().__init__(mean=mean_inv, std=std_inv)

    def __call__(self, tensor):
        return super().__call__(tensor.clone())
    

In [ ]:
class FlowerDataset(Dataset):
    def __init__(self, root: str, jsfile: str, ext: str = "jpg", transform: Optional[Callable] = None):
        self.root = Path(root)
        self.jsfile = Path(jsfile)
        self.ext = ext
        self.transform = transform
        
        self._load_files()
        self._load_classes()
        
    def _load_files(self):
        self.files = sorted(list(self.root.glob(f"*/*.{self.ext}")))
    
    def _load_classes(self):
        cat2name = self._load_json(str(self.jsfile))
        idx2class, class2idx, data_dict = [], {}, {}

        for k,v in cat2name.items(): data_dict[v] = int(k)
        list_pair = sorted(data_dict.items(), key=lambda x: x[1])
        for (name,cat) in list_pair:
            class2idx[name] = cat - 1
            idx2class.append(name)
        
        self.class2idx = class2idx
        self.idx2class = idx2class

    def _load_json(self, path):
        with open(path, 'r') as jsfile:
            data = json.load(jsfile)
        return data
    
    def _load_image(self, path):
        img = Image.open(path)
        return img
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        impath = self.files[idx]
        img = self._load_image(str(impath))
        cat = int(impath.parent.name)
        
        if self.transform:
            img = self.transform(img)

        return img, cat

In [ ]:
NGPU = 1
NUM_EPOCH = 100
BSIZE = 32
LRATE = 0.005
MOMENTUM=0.9

LRSTEP=7
GAMMA=0.1
PRINT_FREQ = 5

In [ ]:
base_dir = Path('/kaggle/input/pytorch-challange-flower-dataset/')
cat2name_path = base_dir.joinpath('cat_to_name.json') 
train_path = base_dir.joinpath('dataset/train')
valid_path = base_dir.joinpath('dataset/valid')

In [ ]:
# TODO: Define your transforms for the training and validation sets
#  scaling, rotations, mirroring, and/or cropping
mean_val, std_val = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
train_transforms =  T.Compose([
    T.RandomRotation(30),
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean_val,std_val)
])

valid_transforms =  T.Compose([
    T.Resize(255),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean_val,std_val)
])

trainset = FlowerDataset(root=train_path, jsfile=cat2name_path, transform=train_transforms)
validset = FlowerDataset(root=valid_path, jsfile=cat2name_path, transform=valid_transforms)

train_loader = DataLoader(trainset, batch_size=BSIZE, shuffle=True, num_workers=0)
valid_loader = DataLoader(validset, batch_size=BSIZE, shuffle=True, num_workers=0)

In [ ]:
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# plot the images in the batch, along with the corresponding labels
denorm = NormalizeInverse(mean_val, std_val)
fig = plt.figure(figsize=(25, 4))
# display 20 images
for idx in np.arange(BSIZE-1):
    ax = fig.add_subplot(3, BSIZE/2, idx+1, xticks=[], yticks=[])
    img = denorm(images[idx]).permute(1,2,0).numpy()
    img = np.clip(img, 0, 1)
    plt.imshow(img, cmap='gray')
    ax.set_title(trainset.idx2class[idx])